In [1]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [2]:
class states(dict):
    paragraph: str
    structured_data:dict | None
    is_chat: bool
    generated_code : str | None

In [4]:
from langchain_groq import ChatGroq
global model
model = ChatGroq(model="llama-3.1-8b-instant",groq_api_key="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")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002902EFBD940>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002902EFBE510>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
def get_the_values(state: states):
    prompt = f"""
            Extract chartable data from this paragraph.
            Respond ONLY in JSON format like this:
            {{
            "x_axis": [...],
            "y_axis": [...],
            "chart_type": "bar|line|pie"
            }}
            If no chart is possible, respond with: "NO_CHART"


            Paragraph: {state['paragraph']}
        """
    res = model.predict(prompt)
    if "NO_CHART" in res:
        state["structured_data"] = None
    else:
        import json
        try:
            state["structured_data"] = json.loads(res)
        except Exception:
            state["structured_data"] = None
    return state

In [45]:
def chart_possible(state: states):
    if state["structured_data"] is None:
        state["is_chat"] = False
    else:
        state["is_chat"] = True

    return state

In [17]:
from langchain_core.prompts import ChatPromptTemplate
def generate_the_plot(state: states):
    template = ChatPromptTemplate.from_messages(
        [
            ("system", """You are a Python data visualization assistant. 
                I will provide:
                - X values
                - Y values
                - A description of the chart type or style I want

                Your task:
                1. Generate valid Python code using matplotlib (and seaborn if needed) to plot the chart.
                2. The code must:
                - Import necessary libraries
                - Plot the chart correctly
                - Add axis labels and a title (if applicable)
                - Call plt.show() at the end
                3. Do not include explanations or comments, only return Python code.
                """),
            ("user", "{message}")
        ]
    )
    chain = template|model
    res = chain.invoke({"message":state["structured_data"]})
    state["generated_code"] = res.content
    return state
    

In [33]:
import ast
def remove_invalid_lines(code):
    valid_lines = []
    for line in code.splitlines():
        try:
            ast.parse(line)
            valid_lines.append(line)
        except SyntaxError:
            pass
    return "\n".join(valid_lines)

In [ ]:
import subprocess
def run_the_code(filename):
    result = subprocess.run(
        ["python", f"./generated_codes/{filename}"],
        capture_output=True
    )
    print(result.stdout)
    print(result.stderr)


In [47]:
# Example paragraph
from langgraph.graph import StateGraph, END, START
import datetime

paragraph = """
    Over the past five years, the adoption of renewable energy has steadily increased, with solar energy showing the fastest growth compared to wind and hydro power. Solar installations have grown at a higher rate year after year, while wind energy has maintained moderate but consistent expansion. Hydropower, on the other hand, has remained relatively stable with only slight improvements. A visualization comparing these three energy sources over time would clearly highlight solar’s rapid rise against the steadier trends of wind and hydro.
"""

# Initial state
state = states(paragraph=paragraph, structured_data=None, is_chat=False)

graph = StateGraph(states)
graph.add_node("extract", get_the_values)
graph.add_node("decide", chart_possible)
graph.add_node("chart", generate_the_plot)

graph.set_entry_point("extract")
graph.add_edge("extract", "decide")
graph.add_conditional_edges(
    "decide",
    lambda state: "chart" if state["is_chat"] else END,
    {
        "chart": "chart",
        END: END
    }
)
graph.add_edge("chart",END)

flow = graph.compile()

res = flow.invoke(state)
output = remove_invalid_lines(res['generated_code'])

if res['generated_code'] is not None:
    filename = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + ".py"
    print(filename)
    with open(f"./generated_codes/{filename}", "w") as f:
        f.writelines(output)
        f.close()

run_the_code(filename)


20250903_003341.py
b'Figure(1000x600)\r\n'
b''
